# Упражнение 1. Удаление элемента из АВЛ-дерева

1. Добавьте метод `verify()` в класс `AVLTree`. Метод должен работать так же, как и метод `verify()` класса `BST`, то есть выбрасывать исключение `BrokenTreeError`, если свойства АВЛ-дерева нарушаются. Можете ориентироваться на реализацию `RBT.verify()` в alg11.ipynb.

- Добавьте метод `remove()` в класс `AVLTree`.

In [4]:
bst = BSTNode(3, 'value')

In [6]:
print(bst)

BSTNode(key=3, value='value', left=None, right=None, parent=None)


In [1]:
class BSTNode:
    # Description of __slots__ attribute can be found at
    # https://pythonz.net/references/named/object.__slots__/
    # Here __slots__ is used in __str__() method.
    __slots__ = ('key', 'value', 'left', 'right', 'parent')
    
    def __init__(self, key, value, left=None, right=None, parent=None):
        self.key = key
        self.value = value
        self.left = left
        self.right = right
        self.parent = parent
        
    def __str__(self):
        print_str = []
        for k in self.__slots__:
            if k in ['left', 'right', 'parent']:
                n = getattr(self, k)
                if n is None:
                    v = None
                else:
                    k += '.key'
                    v = n.key
                print_str.append('{}={}'.format(k, repr(v)))
            else:
                print_str.append('{}={}'.format(k, repr(getattr(self, k))))
        return self.__class__.__name__ + '(' + ', '.join(print_str) + ')'
    
    def is_left_child(self):
        if self.parent is None:
            return False
        if self is self.parent.left:
            return True
        return False
    
    def is_right_child(self):
        if self.parent is None:
            return False
        if self is self.parent.right:
            return True
        return False
    
    def get_attribute_values_string(self, attributes):
        """Returns a string with names and values of
        specified attributes. Used in Tree.prefix_traverse()
        method for optional demostration of node contents."""
        
        values = [getattr(self, attr) for attr in attributes]
        attr_strings = ['{}={}'.format(attr, repr(v)) for attr, v in zip(attributes, values)]
        return ' '.join(attr_strings)
    
    def _get_slots_attributes(self):
        attributes = {}
        for attr in self.__slots__:
            attributes[attr] = getattr(self, attr)
        return attributes
        
    def copy(self):
        kwargs = self._get_slots_attributes()
        return self.__class__(**kwargs)


class BrokenTreeError(Exception):
    def __init__(self, message, node, notional_depth=None):
        super().__init__(message)
        self.node = node
        self.notional_depth = notional_depth
        

class BST:
    def __init__(self):
        self.root = None

    ############################
    # Traverse methods
    
    def _traverse(self, root):
        if root is not None:
            self._traverse(root.left)
            print(root.key, root.value)
            self._traverse(root.right)
    
    def traverse(self):
        """Infix traverse"""
        self._traverse(self.root)
        
    @staticmethod
    def _get_connection_lines_for_children(connections):
        connections = connections.copy()
        if len(connections) == 0:
            return connections
        if connections[-1] == '├':
            connections[-1] = '│'
        elif connections[-1] == '└':
            connections[-1] = ' '
        else:
            raise ValueError('no connection to current node')
        return connections
            
    @staticmethod
    def _print_connections(connections):
        connections_str = ''
        for c in connections:
            if c == ' ':
                connections_str += c*4
            elif c == '│':
                connections_str += c + ' '*3
            elif c in '├└':
                connections_str += c + '─'*2 + ' '
            else:
                raise ValueError("unknown character '{}' in connections list".format(c))
        print(connections_str, end='')
        
    def _prefix_traverse(self, root, connections, attributes):
        self._print_connections(connections)
        connections = self._get_connection_lines_for_children(connections)
        if root is not None:
            print(root.key, root.get_attribute_values_string(attributes))
            connections_right = connections.copy()
            connections_right.append('├')
            self._prefix_traverse(root.right, connections_right, attributes)
            connections_left = connections.copy()
            connections_left.append('└')
            self._prefix_traverse(root.left, connections_left, attributes)
        else:
            print('-')
        
    def prefix_traverse(self, attributes=()):
        """Prefix traverse which draws connections between nodes.
        `attributes` is a tuple containing names of attributes
        of tree node which will be printed during traversing.
        For instance if you need to print values in nodes you
        may use
        ```python
        import random
        for _ in range(5):
            key = random.randint(0, 5)
            bst.insert(key, str(key))
        bst.prefix_traverse(('value',))
        ```
        The output will look similar to
        ```
        8 value='8'
        ├── 12 value='12'
        │   ├── 13 value='13'
        │   │   ├── -
        │   │   └── -
        │   └── 9 value='9'
        │       ├── -
        │       └── -
        └── 3 value='3'
            ├── -
            └── -
        ```
        By default only keys of the node are printed.
        The right subtree is printed before the left subtree.
        If a node has no child, hyphen is printed instead of 
        child's key.
        Args:
            attributes: a tuple of strings
        
        Returns:
            None
        """
        self._prefix_traverse(self.root, [], attributes)
        
    ##################################
    # Insertion methods
        
    def _insert(self, key, value, root):
        if key == root.key:
            root.value = value
        elif key < root.key:
            if root.left is None:
                root.left = BSTNode(key, value, parent=root)
            else:
                self._insert(key, value, root.left)
        else:
            if root.right is None:
                root.right = BSTNode(key, value, parent=root)
            else:
                self._insert(key, value, root.right)
        
    def insert(self, key, value):
        if self.root is None:
            self.root = BSTNode(key, value)
        else:
            self._insert(key, value, self.root)
            
    ###################################
    # Search and removal methods
            
    def _find(self, key, root):
        if root is None:
            return None
        if key == root.key:
            return root  # ATTENTION changed from root.value
        elif key < root.key:
            return self._find(key, root.left)
        else:
            return self._find(key, root.right)
    
    def _remove(self, key, root):
        node = self._find(key, root)
        if node is not None:
            if node.left is None:
                successor = node.right
                self._set_links_with_parent_for_node_replacement(node, successor)
            elif node.right is None:
                successor = node.left
                self._set_links_with_parent_for_node_replacement(node, successor)
            else:
                successor = self._find_min_in_subtree(key, node.right).copy()
                node.key = successor.key
                node.value = successor.value
                self._remove(successor.key, node.right)
        else:
            raise KeyError("node with key {} is not found in BST".format(repr(key)))

    def _find_min_in_subtree(self, key, root):
        if root.left is None:
            return root
        else:
            return self._find_min_in_subtree(key, root.left)
        
    def _set_links_with_parent_for_node_replacement(self, node, successor):
        parent = node.parent
        if parent is None:
            self.root = successor
        else:
            if node.is_left_child():
                parent.left = successor
            else:
                parent.right = successor
        if successor is not None:
            successor.parent = parent
                
    def remove(self, key):
        self._remove(key, self.root)
        
    def find(self, key):
        return self._find(key, self.root).value
    
    ###################################
    # Rotation methods
    
    def _rotate_left(self, node):
        """Left rotation of subtree with root at `node`.
            node                 B
           /    \               / \
          A      B   ===>   node   D
                / \        /    \   
               C   D      A      C
        Args:
            node: an instance of `BSTNode` class or `BSTNode` subclass

        Returns:
            None
        """
        
        r = node.right
        if r is None:
            raise ValueError("no right subtree")
        rl = r.left

        p = node.parent
        
        if p is None:
            self.root = r
        else:
            if node.is_left_child():
                p.left = r
            else:
                p.right = r
        
        r.parent = p
        r.left = node
        
        node.parent = r
        node.right = rl
        
        if rl is not None:
            rl.parent = node
         
    def _rotate_right(self, node):
        """Right rotation of subtree with root at node.
            node                   A
           /    \                 / \
          A      B     ===>      C   node
         / \                        /    \   
        C   D                      D      B
        Args:
            node: an instance of BSTNode class or BSTNode subclass

        Returns:
            None
        """
        
        left = node.left
        if left is None:
            raise ValueError("no left subtree")
        lr = left.right

        p = node.parent
        
        if p is None:
            self.root = left
        else:
            if node.is_left_child():
                p.left = left
            else:
                p.right = left
        
        left.parent = p
        left.right = node
        
        node.parent = left
        node.left = lr
        
        if lr is not None:
            lr.parent = node
            
    ###################################
    # Verification methods
            
    def _verify_links(self):
        if self.root.parent is not None:
            raise BrokenTreeError(
                "the root of the tree has parent"
                "\nself.root={}".format(self.root),
                self.root,
            )
        self._verify_links_recur(self.root)
        
    @staticmethod
    def _check_if_loop(node, direction):
        node_2 = getattr(node, direction)
        if node_2 is node:
            raise BrokenTreeError(
                "{} attribute of the node with key {} "
                "points at the same node {}".format(
                    repr(direction), repr(node.key), node),
                node,
            )
    
    @staticmethod
    def _check_connection(parent, direction):
        child = getattr(parent, direction)
        if child is not None:
            if parent is not child.parent:
                raise BrokenTreeError(
                    "child node {} does not point "
                    "at its parent {}".format(child, parent),
                    parent,
                )
            
    def _verify_links_recur(self, root):
        if root is None:
            return
        self._check_if_loop(root, 'left')
        self._check_if_loop(root, 'right')
        self._check_connection(root, 'left')
        self._check_connection(root, 'right')
        self._verify_links_recur(root.left)
        self._verify_links_recur(root.right)
        
    @staticmethod
    def _fits_in(limits, value):
        return ((limits[0] is None or value > limits[0]) and 
                (limits[1] is None or value < limits[1]))
    
    def _verify_bin(self, root, limits):
        if root is None:
            return
        if not self._fits_in(limits, root.key):
            raise BrokenTreeError(
                "key of node {} does not fit "
                "in limits {}".format(root, repr(limits)),
                root,
            )
        self._verify_bin(root.left, [limits[0], root.key])
        self._verify_bin(root.right, [root.key, limits[1]])
            
    def verify(self):
        """Checks correctness of links between nodes, e.g.
        child parent attribute points at its parent. Verifies 
        that tree is binary search tree.
        """
        
        if self.root is not None:
            self._verify_links()
            self._verify_bin(self.root, [None, None])

In [2]:
class AVLNode(BSTNode):
    def __init__(self, key, value, left=None, right=None, parent=None, balance=0):
        super().__init__(key, value, left=left, right=right, parent=parent)
        self.balance = balance
        
        
class AVLTree(BST):                
    def _get_change_of_height_for_left_rotation(self, node_balance, right_balance):
#         print(node_balance, right_balance)
        l_height = 0
        r_height = node_balance
        rr_height = r_height - 1 if right_balance > 0 else r_height - 1 + right_balance
        rl_height = r_height - 1 if right_balance < 0 else r_height - 1 - right_balance
        old_height = max(l_height, r_height) + 1
        new_height = max(l_height+2, rl_height+2, rr_height+1)
        return new_height - old_height
    
    def _rotate_left(self, node):
        height_change = self._get_change_of_height_for_left_rotation(node.balance, node.right.balance)
        node.balance = node.balance - max(node.right.balance, 0) - 1
        node.right.balance = node.right.balance + min(node.balance, 0) - 1
        super()._rotate_left(node)
        return height_change
        
    def _rotate_right(self, node):
        height_change = self._get_change_of_height_for_left_rotation(-node.balance, -node.left.balance)
        node.balance = node.balance - min(node.left.balance, 0) + 1
        node.left.balance = node.left.balance + max(node.balance, 0) + 1
        super()._rotate_right(node)
        return height_change
    
    def _balance_unbalanced_node(self, node):
        assert node.balance > 1 or node.balance < -1, "node does not need balancing\n{}".format(node)
        if node.balance == -2:
            left = node.left
            assert left.balance != 0, \
                "tree was unbalanced before operation\nnode:\n{}\nleft child:\n{}".format(node, left)
            assert -3 < left.balance < 3, \
                "unsupported balance value in node\n{}".format(left)
            if left.balance == -1:
                height_change = self._rotate_right(node)
                assert height_change == -1, \
                    "unexpected height change during balance fixing\nheight_change={}".format(height_change)
            else:
                height_change = self._rotate_left(left)
                assert height_change == 0, \
                    "height change is not zero on first stage of big rotation\n" \
                    "height_change={}".format(height_change)
                height_change = self._rotate_right(node)
                assert height_change == -1, \
                    "unexpected height change during balance fixing\nheight_change={}".format(height_change)
        else:
            right = node.right
            assert right.balance != 0, \
                "tree was unbalanced before operation\nnode:\n{}\right child:\n{}".format(node, right)
            assert -3 < right.balance < 3, \
                "unsupported balance value in node\n{}".format(right)
            if right.balance == 1:
                height_change = self._rotate_left(node)
                assert height_change == -1, \
                    "unexpected height change during balance fixing\nheight_change={}".format(height_change)
            else:
                height_change = self._rotate_right(right)
                self._prefix_traverse(right.parent, [], ['balance'])
                assert height_change == 0, \
                    "height change is not zero on first stage of big rotation\n" \
                    "height_change={}".format(height_change)
                height_change = self._rotate_left(node)
                assert height_change == -1, \
                    "unexpected height change during balance fixing\nheight_change={}".format(height_change)
    
    def _retrace_insert_my(self, node):
        parent = node.parent
        if parent is None:
            return
        assert -2 < parent.balance < 2, "tree was unbalanced before insertion\nbalance={}".format(parent.balance)
        if node.is_left_child():
            parent.balance -= 1
            if parent.balance == 0:
                return
            if parent.balance == -1:
                self._retrace_insert_my(parent)
            else:
                self._balance_unbalanced_node(parent)
        else:
            parent.balance += 1
            if parent.balance == 0:
                return
            if parent.balance == 1:
                self._retrace_insert_my(parent)
            else:
                self._balance_unbalanced_node(parent)
                
    
    def _insert(self, key, value, root):
        if key == root.key:
            root.value = value
        elif key < root.key:
            if root.left is None:
                root.left = AVLNode(key, value, parent=root)
                self._retrace_insert_my(root.left)
            else:
                self._insert(key, value, root.left)
        else:
            if root.right is None:
                root.right = AVLNode(key, value, parent=root)
                self._retrace_insert_my(root.right)
            else:
                self._insert(key, value, root.right)
    
    def insert(self, key, value):
        if self.root is None:
            self.root = AVLNode(key, value)
        else:
            self._insert(key, value, self.root)
            
            
            
    ############################
    # Verification methods
        
    
    def _verify(self, root):
        if root is None:
            return 1
        self._verify(root.left)
        self._verify(root.right)
        if root.balance not in [0, 1, -1]:
            raise BrokenTreeError(
                "Not balanced",
                root
            )
            
    def verify(self):
        """Verifies that red-black tree properties"""
        
        super().verify()
        _ = self._verify(self.root)

In [17]:
avl = AVLTree()

for i in range(100):
    i = random.randint(0, 1000)
    
    avl.insert(i, str(i))
    # avl.prefix_traverse(['balance'])
    avl.verify()
    
# avl.prefix_traverse(['balance'])

106 balance=1
├── 149 balance=0
│   ├── -
│   └── -
└── -
721 balance=1
├── 911 balance=1
│   ├── 914 balance=1
│   │   ├── 929 balance=0
│   │   │   ├── -
│   │   │   └── -
│   │   └── -
│   └── 749 balance=0
│       ├── -
│       └── -
└── 701 balance=-1
    ├── -
    └── 654 balance=0
        ├── -
        └── -
825 balance=1
├── 864 balance=0
│   ├── -
│   └── -
└── -
222 balance=1
├── 265 balance=0
│   ├── -
│   └── -
└── -
242 balance=1
├── 265 balance=1
│   ├── 274 balance=0
│   │   ├── -
│   │   └── -
│   └── -
└── 239 balance=0
    ├── -
    └── -
592 balance=1
├── 721 balance=1
│   ├── 911 balance=-1
│   │   ├── 914 balance=1
│   │   │   ├── 929 balance=0
│   │   │   │   ├── -
│   │   │   │   └── -
│   │   │   └── -
│   │   └── 825 balance=-1
│   │       ├── 864 balance=0
│   │       │   ├── -
│   │       │   └── -
│   │       └── 749 balance=1
│   │           ├── 824 balance=0
│   │           │   ├── -
│   │           │   └── -
│   │           └── -
│   └── 654 balance=0
│  

In [3]:
import random

avl = AVLTree()
for i in [0, 1, 4, 3, 5, 2]:
    print('*********')
    print('INSERTING', i)
    avl.insert(i, str(i))
    avl.prefix_traverse(['balance'])
print('#######################################')

for i in range(100):
    i = random.randint(0, 1000)
    
    print('*********')
    print('INSERTING', i)
    avl.insert(i, str(i))
    avl.prefix_traverse(['balance'])
    
avl.prefix_traverse()

avl._rotate_right(avl.root)    
print('*************************')
avl.prefix_traverse()

*********
INSERTING 0
0 balance=0
├── -
└── -
*********
INSERTING 1
0 balance=1
├── 1 balance=0
│   ├── -
│   └── -
└── -
*********
INSERTING 4
1 balance=0
├── 4 balance=0
│   ├── -
│   └── -
└── 0 balance=0
    ├── -
    └── -
*********
INSERTING 3
1 balance=1
├── 4 balance=-1
│   ├── -
│   └── 3 balance=0
│       ├── -
│       └── -
└── 0 balance=0
    ├── -
    └── -
*********
INSERTING 5
1 balance=1
├── 4 balance=0
│   ├── 5 balance=0
│   │   ├── -
│   │   └── -
│   └── 3 balance=0
│       ├── -
│       └── -
└── 0 balance=0
    ├── -
    └── -
*********
INSERTING 2
3 balance=1
├── 4 balance=1
│   ├── 5 balance=0
│   │   ├── -
│   │   └── -
│   └── -
└── 2 balance=0
    ├── -
    └── -
3 balance=0
├── 4 balance=1
│   ├── 5 balance=0
│   │   ├── -
│   │   └── -
│   └── -
└── 1 balance=0
    ├── 2 balance=0
    │   ├── -
    │   └── -
    └── 0 balance=0
        ├── -
        └── -
#######################################
*********
INSERTING 782
3 balance=0
├── 5 balance=0
│   ├── 782

        ├── 2 balance=0
        │   ├── -
        │   └── -
        └── 0 balance=0
            ├── -
            └── -
*********
INSERTING 155
118 balance=1
├── 477 balance=0
│   ├── 782 balance=0
│   │   ├── 915 balance=1
│   │   │   ├── 995 balance=0
│   │   │   │   ├── -
│   │   │   │   └── -
│   │   │   └── -
│   │   └── 579 balance=0
│   │       ├── 705 balance=0
│   │       │   ├── -
│   │       │   └── -
│   │       └── 552 balance=0
│   │           ├── -
│   │           └── -
│   └── 264 balance=0
│       ├── 450 balance=-1
│       │   ├── -
│       │   └── 289 balance=0
│       │       ├── -
│       │       └── -
│       └── 206 balance=0
│           ├── 246 balance=0
│           │   ├── -
│           │   └── -
│           └── 155 balance=0
│               ├── -
│               └── -
└── 3 balance=0
    ├── 5 balance=0
    │   ├── 85 balance=0
    │   │   ├── -
    │   │   └── -
    │   └── 4 balance=0
    │       ├── -
    │       └── -
    └── 1 balance=0
        ├── 2 bala

├── 579 balance=0
│   ├── 782 balance=0
│   │   ├── 915 balance=1
│   │   │   ├── 981 balance=0
│   │   │   │   ├── 995 balance=0
│   │   │   │   │   ├── -
│   │   │   │   │   └── -
│   │   │   │   └── 955 balance=0
│   │   │   │       ├── -
│   │   │   │       └── -
│   │   │   └── 881 balance=0
│   │   │       ├── -
│   │   │       └── -
│   │   └── 705 balance=1
│   │       ├── 736 balance=1
│   │       │   ├── 779 balance=0
│   │       │   │   ├── -
│   │       │   │   └── -
│   │       │   └── -
│   │       └── 634 balance=0
│   │           ├── -
│   │           └── -
│   └── 477 balance=-1
│       ├── 552 balance=-1
│       │   ├── -
│       │   └── 543 balance=0
│       │       ├── -
│       │       └── -
│       └── 450 balance=-1
│           ├── 473 balance=0
│           │   ├── -
│           │   └── -
│           └── 289 balance=1
│               ├── 426 balance=0
│               │   ├── -
│               │   └── -
│               └── -
└── 5 balance=0
    ├── 118 balance=0
 

│               │   └── -
│               └── -
└── 5 balance=1
    ├── 118 balance=1
    │   ├── 206 balance=-1
    │   │   ├── 246 balance=0
    │   │   │   ├── -
    │   │   │   └── -
    │   │   └── 155 balance=0
    │   │       ├── 193 balance=0
    │   │       │   ├── -
    │   │       │   └── -
    │   │       └── 132 balance=0
    │   │           ├── -
    │   │           └── -
    │   └── 58 balance=0
    │       ├── 85 balance=0
    │       │   ├── -
    │       │   └── -
    │       └── 32 balance=0
    │           ├── -
    │           └── -
    └── 3 balance=-1
        ├── 4 balance=0
        │   ├── -
        │   └── -
        └── 1 balance=0
            ├── 2 balance=0
            │   ├── -
            │   └── -
            └── 0 balance=0
                ├── -
                └── -
*********
INSERTING 712
264 balance=0
├── 579 balance=0
│   ├── 782 balance=0
│   │   ├── 955 balance=0
│   │   │   ├── 981 balance=0
│   │   │   │   ├── 995 balance=0
│   │   │   │   │   ├──

│       │   └── 505 balance=0
│       │       ├── -
│       │       └── -
│       └── 450 balance=-1
│           ├── 473 balance=0
│           │   ├── -
│           │   └── -
│           └── 289 balance=1
│               ├── 426 balance=0
│               │   ├── -
│               │   └── -
│               └── -
└── 5 balance=1
    ├── 118 balance=1
    │   ├── 206 balance=-1
    │   │   ├── 246 balance=0
    │   │   │   ├── -
    │   │   │   └── -
    │   │   └── 155 balance=0
    │   │       ├── 193 balance=0
    │   │       │   ├── -
    │   │       │   └── -
    │   │       └── 132 balance=0
    │   │           ├── -
    │   │           └── -
    │   └── 58 balance=0
    │       ├── 85 balance=0
    │       │   ├── -
    │       │   └── -
    │       └── 32 balance=0
    │           ├── -
    │           └── -
    └── 3 balance=-1
        ├── 4 balance=0
        │   ├── -
        │   └── -
        └── 1 balance=0
            ├── 2 balance=0
            │   ├── -
            │   └── 

...

    │       │   │   │   └── -
    │       │   │   └── -
    │       │   └── 322 balance=0
    │       │       ├── -
    │       │       └── -
    │       └── 273 balance=1
    │           ├── 276 balance=0
    │           │   ├── -
    │           │   └── -
    │           └── -
    └── 58 balance=1
        ├── 118 balance=1
        │   ├── 193 balance=0
        │   │   ├── 244 balance=0
        │   │   │   ├── 246 balance=1
        │   │   │   │   ├── 250 balance=0
        │   │   │   │   │   ├── -
        │   │   │   │   │   └── -
        │   │   │   │   └── -
        │   │   │   └── 206 balance=0
        │   │   │       ├── 215 balance=0
        │   │   │       │   ├── -
        │   │   │       │   └── -
        │   │   │       └── 200 balance=0
        │   │   │           ├── -
        │   │   │           └── -
        │   │   └── 155 balance=0
        │   │       ├── 192 balance=-1
        │   │       │   ├── -
        │   │       │   └── 187 balance=0
        │   │       │       

        ├── 118 balance=1
        │   ├── 193 balance=0
        │   │   ├── 244 balance=0
        │   │   │   ├── 246 balance=1
        │   │   │   │   ├── 250 balance=0
        │   │   │   │   │   ├── -
        │   │   │   │   │   └── -
        │   │   │   │   └── -
        │   │   │   └── 206 balance=0
        │   │   │       ├── 215 balance=0
        │   │   │       │   ├── -
        │   │   │       │   └── -
        │   │   │       └── 200 balance=0
        │   │   │           ├── -
        │   │   │           └── -
        │   │   └── 155 balance=0
        │   │       ├── 192 balance=-1
        │   │       │   ├── -
        │   │       │   └── 187 balance=0
        │   │       │       ├── -
        │   │       │       └── -
        │   │       └── 132 balance=1
        │   │           ├── 154 balance=0
        │   │           │   ├── -
        │   │           │   └── -
        │   │           └── -
        │   └── 73 balance=0
        │       ├── 90 balance=0
        │       │   ├

        │   │   │   │   └── -
        │   │   │   └── -
        │   │   └── 206 balance=0
        │   │       ├── 215 balance=-1
        │   │       │   ├── -
        │   │       │   └── 208 balance=0
        │   │       │       ├── -
        │   │       │       └── -
        │   │       └── 200 balance=1
        │   │           ├── 204 balance=0
        │   │           │   ├── -
        │   │           │   └── -
        │   │           └── -
        │   └── 118 balance=0
        │       ├── 155 balance=0
        │       │   ├── 187 balance=0
        │       │   │   ├── 192 balance=0
        │       │   │   │   ├── -
        │       │   │   │   └── -
        │       │   │   └── 186 balance=0
        │       │   │       ├── -
        │       │   │       └── -
        │       │   └── 132 balance=1
        │       │       ├── 154 balance=0
        │       │       │   ├── -
        │       │       │   └── -
        │       │       └── -
        │       └── 73 balance=0
        │           

    └── 58 balance=1
        ├── 193 balance=0
        │   ├── 244 balance=-1
        │   │   ├── 246 balance=1
        │   │   │   ├── 250 balance=0
        │   │   │   │   ├── -
        │   │   │   │   └── -
        │   │   │   └── -
        │   │   └── 206 balance=0
        │   │       ├── 215 balance=-1
        │   │       │   ├── -
        │   │       │   └── 208 balance=0
        │   │       │       ├── -
        │   │       │       └── -
        │   │       └── 200 balance=1
        │   │           ├── 204 balance=0
        │   │           │   ├── -
        │   │           │   └── -
        │   │           └── -
        │   └── 118 balance=0
        │       ├── 155 balance=0
        │       │   ├── 187 balance=0
        │       │   │   ├── 192 balance=0
        │       │   │   │   ├── -
        │       │   │   │   └── -
        │       │   │   └── 186 balance=0
        │       │   │       ├── -
        │       │   │       └── -
        │       │   └── 132 balance=1
        │    

│           │   ├── 394 
│           │   │   ├── 404 
│           │   │   │   ├── -
│           │   │   │   └── -
│           │   │   └── 387 
│           │   │       ├── -
│           │   │       └── -
│           │   └── 322 
│           │       ├── -
│           │       └── -
│           └── 273 
│               ├── 276 
│               │   ├── -
│               │   └── -
│               └── -
└── 58 
    ├── 193 
    │   ├── 244 
    │   │   ├── 246 
    │   │   │   ├── 250 
    │   │   │   │   ├── -
    │   │   │   │   └── -
    │   │   │   └── -
    │   │   └── 206 
    │   │       ├── 215 
    │   │       │   ├── 238 
    │   │       │   │   ├── -
    │   │       │   │   └── -
    │   │       │   └── 208 
    │   │       │       ├── -
    │   │       │       └── -
    │   │       └── 200 
    │   │           ├── 204 
    │   │           │   ├── -
    │   │           │   └── -
    │   │           └── -
    │   └── 118 
    │       ├── 155 
    │       │   ├── 187 
    │       │  